**Install Packages**

In [1]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

**Functions**

In [2]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

**Locals & Constants**

In [3]:
############
# OPTIONAL #
############

# mount drive
#drive.mount("/content/drive", force_remount=False)

# data location
file_dir = "/content/drive/My Drive/Colab Data/input/" # optional



# New Section

In [4]:
# read in api key file
#df_api_keys = pd.read_csv(file_dir + "api_keys.csv")
# get keys
#api_key = df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key
#api_key ='156b46f9-1d41-40b6-8052-62510a6d3327
api_key ='bb649b92-e19a-4bd2-b5aa-37df638575e8'

**Data**

**1. Property Listings For Sale**

In [5]:
## zillow search url
#listing_url = "https://www.zillow.com/prosper-tx/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A45.64974638372341%2C%22south%22%3A27.61198105167906%2C%22east%22%3A-71.76382025000001%2C%22west%22%3A-98.83413275000001%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A33547%2C%22regionType%22%3A6%7D%2C%7B%22regionId%22%3A6181%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A5%7D"
listing_url = "https://www.zillow.com/homes/for_sale/?searchQueryState=%7B%22mapBounds%22%3A%7B%22north%22%3A40.896601543545266%2C%22east%22%3A-73.73176979892379%2C%22south%22%3A40.68137423871844%2C%22west%22%3A-74.1334574209941%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22days%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%2C%22customRegionId%22%3A%22a1905651baX1-CR1d5cdoyspx32d_1ec136%22%7D"
# get listings
listing_response = get_listings(api_key, listing_url)

In [6]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

KeyError: ignored

In [ ]:
import json
x=json.dumps( listing_response.json())
print(x)
with open('x','w') as outfile:
  outfile.write(x)


In [ ]:
# view all listings
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

In [ ]:
df_listings.to_csv("manhattan_listings.csv")
files.download("manhattan_listings.csv")

**Rental Property Listings For Rent**

In [ ]:
# zillow search url
rent_listing_url = "https://www.zillow.com/homes/for_rent/?searchQueryState=%7B%22mapBounds%22%3A%7B%22north%22%3A40.812333570567255%2C%22east%22%3A-73.91048556073008%2C%22south%22%3A40.75852378661119%2C%22west%22%3A-74.01090746624766%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22days%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%2C%22customRegionId%22%3A%22a1905651baX1-CR1d5cdoyspx32d_1ec136%22%2C%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22%22%7D"

In [ ]:
# get listings
rent_listing_response = get_listings(api_key, rent_listing_url)

In [ ]:
# view rent zestimate
px.box(df_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")


In [ ]:
listing_response=get_property_detail(api_key, 182378290)

In [ ]:
# view all keys
print(rent_listing_response.json().keys())

# check if request is successful
print("Request success:", rent_listing_response.json()["is_success"])

# view count of properies returned in request
num_of_properties = rent_listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

In [ ]:
# view all listings
df_rent_listings = pd.json_normalize(rent_listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_rent_listings))
print("Number of columns:", len(df_rent_listings.columns))
df_rent_listings

**Get Property price History Data by zpid for all in property listing dataframe**

In [ ]:
from pandas.core.indexes.base import is_string_dtype
df = pd.DataFrame() #Initialise a blank dataframe
#Use count to keep track on how many data fetches are done
count=0
#loop through property listing data and fetch corresponding property details data for each zpid
for zpid in df_listings['zpid']:

    if type(zpid)==str:
      zpid_=zpid

      prop_detail_response = get_property_detail(api_key, zpid_)

      df_prop = pd.json_normalize(prop_detail_response.json()['data'])

      df_price_hist = pd.DataFrame(df_prop["priceHistory"].iloc[0])
      # check if request is successful
      print("Request success:", prop_detail_response.json()["is_success"])
      df_price_hist["zpid"]=zpid_


      if df.empty:
        #Assign df to first property details data captured
        df=df_price_hist

      else:
        #Merge the rest of property details data collected with initial data
        df=df.append(df_price_hist)
        count+=1


**Download price history data**

In [ ]:
df.to_csv("df_price_hist.csv")
files.download("df_price_hist.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>